In [200]:
import pandas as pd
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError
import gspread

%load_ext autoreload
%autoreload 2

# search_value = 'KCB W1Aug25 Fushi - EN'
search_value = [
    "KCB W5Jul25 Karma Subito App Version",
    "KCI W5Jul25 Karma Subito App Version",
    "KCB W5Jul25 Karma Subito App Version - Part 2",
    "KCI W5Jul25 Karma Subito App Version - Part 2",
    "KCB W5Jul25 Karma Subito App Version - Part 3",
    "KCI W5Jul25 Karma Subito App Version - Part 3",
    "KCB W5Jul25 Karma Subito App Version - Part 4",
    "KCI W5Jul25 Karma Subito App Version - Part 4",
    "KCB W1Aug25 Karma Subito App Version - Part 5",
    "KCI W1Aug25 Karma Subito App Version - Part 5",
    "KCB W1Aug25 Karma Subito App Version - Part 6",
    "KCI W1Aug25 Karma Subito App Version - Part 6",
    "KCB W1Aug25 Karma Subito App Version - Part 7",
    "KCI W1Aug25 Karma Subito App Version - Part 7",
    "KCB W1Aug25 Karma Subito App Version - Part 8",
    "KCI W1Aug25 Karma Subito App Version - Part 8",
    "KCB W2Aug25 Karma Subito App Version - Part 9",
    "KCI W2Aug25 Karma Subito App Version - Part 9",
    "KCB W2Aug25 Karma Subito App Version - Part 10",
    "KCI W2Aug25 Karma Subito App Version - Part 10",
    "KCB W2Aug25 Karma Subito App Version - Part 11",
    "KCI W2Aug25 Karma Subito App Version - Part 11",
    "KCB W2Aug25 Karma Subito App Version - Part 12",
    "KCI W2Aug25 Karma Subito App Version - Part 12"
    ]

file_name = '\mailchimp_report_details'
file_type = '.xlsx'
file_loc = r'C:\Users\fajar.fatoni\Documents\Python\Data'

file_path = file_loc+file_name+file_type

# Replace with your actual API key and client ID
api_key = '4af88cb8f553cbf45b6a5de8bcb88a5f'
server = 'us17'

# campaign_id = '0a64cef1d9'

# API endpoint
# url = f'https://api.createsend.com/api/v3.2/clients/{client_id}/campaigns.json'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [201]:
gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1zk0XUrqFuL6yb-7oBpjDrLp01-AGmWHBhdzNXg-lBjs'
SHEET_NAME = 'Master EDM'

In [202]:
# Get data from Google Sheets
spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)

In [203]:
def getReportDetail(api_key, server, campaign_id):
    try:
        client = MailchimpMarketing.Client()
        client.set_config({
            "api_key": api_key,
            "server": server
        })

        # Get the campaign report which contains bounce and other details
        response = client.reports.get_campaign_report(campaign_id)
        
        # Corrected extraction logic based on your provided report dictionary
        extracted_data = {
            'Send Time': response.get('send_time'),
            'campaign_id': response.get('id'),
            'Campaign Names': response.get('campaign_title'),
            'Subject Line': response.get('subject_line'),
            'Total Email Sent': response.get('emails_sent'),
            'Opens': response.get('opens', {}).get('opens_total', 0),
            'Unique Opens': response.get('opens', {}).get('unique_opens', 0),
            'Clicks': response.get('clicks', {}).get('clicks_total', 0),
            'Unique Clicks': response.get('clicks', {}).get('unique_subscriber_clicks', 0),
            'Unsubscribed': response.get('unsubscribed', 0),
            'Bounced': response.get('bounces', {}).get('hard_bounces', 0) + 
                       response.get('bounces', {}).get('soft_bounces', 0) + 
                       response.get('bounces', {}).get('syntax_errors', 0)
        }

        df_normalized = pd.DataFrame([extracted_data])
        
        # Display the DataFrame
        # display(df_normalized)  # For Jupyter Notebook or similar
        # print(response)      # For standard console output
        
        return df_normalized
    except ApiClientError as error:
        print(f"Error: {error.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [204]:
def get_spam_count(api_key, server, campaign_id):
    """
    Retrieves the spam report count for a given Mailchimp campaign.

    Args:
        api_key (str): Your Mailchimp API key.
        server_prefix (str): Your server prefix (e.g., 'us17').
        campaign_id (str): The ID of the campaign.

    Returns:
        int: The number of abuse reports (spam complaints), or -1 if an error occurs.
    """
    try:
        client = MailchimpMarketing.Client()
        client.set_config({
            "api_key": api_key,
            "server": server
        })

        # Get the campaign report
        response = client.reports.get_campaign_report(campaign_id)

        # The spam count is stored under the 'abuse_reports' key
        spam_count = response.get('abuse_reports', 0)

        # print(f"Spam count for campaign ID '{campaign_id}': {spam_count}")
        return spam_count

    except ApiClientError as error:
        print(f"Error: {error.text}")
        return -1
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return -1

# # Example usage (replace with your actual API key, server prefix, and campaign ID)
# my_api_key = api_key
# my_server_prefix = server
# my_campaign_id = "33d24e12cb"

# count = get_spam_count(my_api_key, my_server_prefix, my_campaign_id)

In [205]:
def getCampaignDetails(api_key, server, search_value):
    import mailchimp_marketing as MailchimpMarketing
    from mailchimp_marketing.api_client import ApiClientError
    
    try:
        client = MailchimpMarketing.Client()
        client.set_config({
            "api_key": api_key,
            "server": server
        })

        response = client.searchCampaigns.search(search_value)
        # print(response) # Keep this if you want to see the raw response

        data = response

        # Initialize lists to collect DataFrames
        extracted_campaign_data = []
        all_dfs = []

        for campaign_info in data['results']:
            campaign = campaign_info['campaign']
            campaign_id = campaign['id']
            campaign_title = campaign['settings']['title']

            # 1. Collect basic campaign info
            extracted_campaign_data.append({'id': campaign_id, 'title': campaign_title})

            # 2. Get Report Detail DataFrame
            current_df = getReportDetail(api_key, server, campaign_id)
            current_df['Mark Spam'] = get_spam_count(api_key, server, campaign_id)

            # Add campaign_name column to the current openers DataFrame
            # current_df['campaign_name'] = campaign_title
            all_dfs.append(current_df) # Add to the list

        # --- Concatenate all collected DataFrames outside the loop ---
        df_campaign = pd.DataFrame(extracted_campaign_data)

        # Concatenate only if lists are not empty to avoid errors on empty searches
        df_report = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame(columns=['Campaign Names'])

        # # Display results
        # print("\n--- Campaign ---")    
        # display(df_campaign)

        # print("\n--- Report ---")
        # display(df_report)

        return df_report # Or return all DataFrames if needed
    except ApiClientError as error:
        print(f"Error: {error.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [206]:
# getOpeners(api_key, server, "b59db76d23")
# getReportDetail(api_key, server, "0a64cef1d9")

dfs_list = []

# Loop through the search_value list
for value in search_value:
    # Call the function for each value and get the dataframe
    df = getCampaignDetails(api_key, server, value)
    
    # Append the resulting dataframe to the list
    dfs_list.append(df)

# Concatenate all dataframes in the list into a single dataframe
df_campaign_final = pd.concat(dfs_list, ignore_index=True)
df_campaign_final.drop_duplicates(subset=['Campaign Names'], inplace=True)

# df_campaign_final = getCampaignDetails(api_key, server, "KCB W2Aug25 Karma Community 290 ID")

# df_campaign_final.to_excel(file_path, index=False)
df_campaign_final.dtypes

Send Time           object
campaign_id         object
Campaign Names      object
Subject Line        object
Total Email Sent     int64
Opens                int64
Unique Opens         int64
Clicks               int64
Unique Clicks        int64
Unsubscribed         int64
Bounced              int64
Mark Spam            int64
dtype: object

In [207]:
float_to_int_columns = [
    'Total Email Sent',
    'Opens',
    'Unique Opens',
    'Clicks',
    'Unique Clicks',
    'Unsubscribed',
    'Bounced',
    'Mark Spam'
]

# Convert the specified columns
for col in float_to_int_columns:
    if col in df_campaign_final.columns: # Check if the column exists in the DataFrame
        # Use .astype(int) to convert float to integer
        # Note: This will truncate decimal values (e.g., 5.9 becomes 5).
        # If there are NaN values, you might need to fill them or use pd.Int64Dtype()
        df_campaign_final[col] = df_campaign_final[col].astype(int)
    else:
        print(f"Column '{col}' not found in the DataFrame.")

df_campaign_final.dtypes

Send Time           object
campaign_id         object
Campaign Names      object
Subject Line        object
Total Email Sent     int64
Opens                int64
Unique Opens         int64
Clicks               int64
Unique Clicks        int64
Unsubscribed         int64
Bounced              int64
Mark Spam            int64
dtype: object

In [208]:
df_campaign_final['Send Time'] = pd.to_datetime(df_campaign_final['Send Time'])
df_campaign_final = df_campaign_final.sort_values(by=['Send Time'])
df_campaign_final['Send Time'] = df_campaign_final['Send Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_campaign_final

,Send Time,campaign_id,Campaign Names,Subject Line,Total Email Sent,Opens,Unique Opens,Clicks,Unique Clicks,Unsubscribed,Bounced,Mark Spam
0,2025-07-29 11:00:00,231b9dfaa2,KCB W5Jul25 Karma Subito App Version,Update Now for Big Karma Energy,577,581,310,67,24,1,0,0
4,2025-07-29 11:15:00,1c0138af9d,KCI W5Jul25 Karma Subito App Version,Update Now for Big Karma Energy,1396,758,490,44,19,1,1,0
1,2025-07-31 11:00:00,3a09ce9393,KCB W5Jul25 Karma Subito App Version - Part 2,Update Now for Big Karma Energy,739,853,461,101,56,2,0,1
5,2025-07-31 11:00:00,d6a72619f4,KCI W5Jul25 Karma Subito App Version - Part 2,Update Now for Big Karma Energy,1236,773,497,26,16,0,4,0
3,2025-08-01 13:30:00,a8c5219d0d,KCB W5Jul25 Karma Subito App Version - Part 3,Update Now for Big Karma Energy,705,720,410,38,27,1,0,0
7,2025-08-01 13:30:00,885b66ae0d,KCI W5Jul25 Karma Subito App Version - Part 3,Update Now for Big Karma Energy,1285,832,507,27,18,0,1,0
2,2025-08-02 13:30:00,35908c48d7,KCB W5Jul25 Karma Subito App Version - Part 4,Update Now for Big Karma Energy,652,666,394,39,26,0,1,0
6,2025-08-02 13:30:00,c009c275f2,KCI W5Jul25 Karma Subito App Version - Part 4,Update Now for Big Karma Energy,1118,641,416,76,12,0,1,0
14,2025-08-05 09:30:00,0901d121e2,KCB W1Aug25 Karma Subito App Version - Part 5,Update Now for Big Karma Energy,551,541,297,15,12,1,0,0
15,2025-08-05 13:30:00,ebfd5d7f34,KCI W1Aug25 Karma Subito App Version - Part 5,Update Now for Big Karma Energy,1377,672,461,12,10,1,1,0


In [209]:
display(df_campaign_final)
# Update keseluruhan data, delete data yang lama terlebih dahulu
# clear all row before update
worksheet.clear()
worksheet.update([df_campaign_final.columns.values.tolist()] + df_campaign_final.values.tolist())

,Send Time,campaign_id,Campaign Names,Subject Line,Total Email Sent,Opens,Unique Opens,Clicks,Unique Clicks,Unsubscribed,Bounced,Mark Spam
0,2025-07-29 11:00:00,231b9dfaa2,KCB W5Jul25 Karma Subito App Version,Update Now for Big Karma Energy,577,581,310,67,24,1,0,0
4,2025-07-29 11:15:00,1c0138af9d,KCI W5Jul25 Karma Subito App Version,Update Now for Big Karma Energy,1396,758,490,44,19,1,1,0
1,2025-07-31 11:00:00,3a09ce9393,KCB W5Jul25 Karma Subito App Version - Part 2,Update Now for Big Karma Energy,739,853,461,101,56,2,0,1
5,2025-07-31 11:00:00,d6a72619f4,KCI W5Jul25 Karma Subito App Version - Part 2,Update Now for Big Karma Energy,1236,773,497,26,16,0,4,0
3,2025-08-01 13:30:00,a8c5219d0d,KCB W5Jul25 Karma Subito App Version - Part 3,Update Now for Big Karma Energy,705,720,410,38,27,1,0,0
7,2025-08-01 13:30:00,885b66ae0d,KCI W5Jul25 Karma Subito App Version - Part 3,Update Now for Big Karma Energy,1285,832,507,27,18,0,1,0
2,2025-08-02 13:30:00,35908c48d7,KCB W5Jul25 Karma Subito App Version - Part 4,Update Now for Big Karma Energy,652,666,394,39,26,0,1,0
6,2025-08-02 13:30:00,c009c275f2,KCI W5Jul25 Karma Subito App Version - Part 4,Update Now for Big Karma Energy,1118,641,416,76,12,0,1,0
14,2025-08-05 09:30:00,0901d121e2,KCB W1Aug25 Karma Subito App Version - Part 5,Update Now for Big Karma Energy,551,541,297,15,12,1,0,0
15,2025-08-05 13:30:00,ebfd5d7f34,KCI W1Aug25 Karma Subito App Version - Part 5,Update Now for Big Karma Energy,1377,672,461,12,10,1,1,0


{'spreadsheetId': '1zk0XUrqFuL6yb-7oBpjDrLp01-AGmWHBhdzNXg-lBjs',
 'updatedRange': "'Master EDM'!A1:L25",
 'updatedRows': 25,
 'updatedColumns': 12,
 'updatedCells': 300}

In [210]:
# import mailchimp_marketing as MailchimpMarketing
# from mailchimp_marketing.api_client import ApiClientError

# try:
#   client = MailchimpMarketing.Client()
#   client.set_config({
#     "api_key": "4af88cb8f553cbf45b6a5de8bcb88a5f",
#     "server": "us17"
#   })

#   response = client.searchCampaigns.search(search_value)
#   print(response)
#   # Check the type of the response object
#   print(f"\nType of response object: {type(response)}")
# except ApiClientError as error:
#   print("Error: {}".format(error.text))

# response


In [211]:
# def getUnsubsDetail(api_key, server, campaign_id):

#   import mailchimp_marketing as MailchimpMarketing
#   from mailchimp_marketing.api_client import ApiClientError

#   try:
#     client = MailchimpMarketing.Client()
#     client.set_config({
#       "api_key": api_key,
#       "server": server
#     })

#     response = client.reports.get_unsubscribed_list_for_campaign(campaign_id)
#     print(response)
#   except ApiClientError as error:
#     print("Error: {}".format(error.text))

#   # Your provided JSON data for unsubscribes
#   unsub_data = response

#   # Extract the list of unsubscribed members
#   unsubscribed_members = unsub_data.get("unsubscribes", [])

#   # Convert the list of dictionaries directly into a DataFrame
#   # pandas can automatically flatten the top-level keys within each dictionary
#   df_unsubs = pd.DataFrame(unsubscribed_members)

#   # Select and display only the 'email_address' and 'reason' (or other desired columns)
#   # You might not need all columns like 'merge_fields', 'vip', etc.
#   df_email_unsubs = df_unsubs[['email_address', 'reason', 'timestamp', 'campaign_id']].copy() if not df_unsubs.empty else pd.DataFrame(columns=['email_address', 'reason', 'timestamp', 'campaign_id'])

#   # print("DataFrame of Unsubscribed Emails:")
#   # display(df_email_unsubs)
#   return df_email_unsubs

In [212]:

# def getOpeners(api_key, server, campaign_id):
#     import mailchimp_marketing as MailchimpMarketing
#     from mailchimp_marketing.api_client import ApiClientError
#     # setup client
#     client = MailchimpMarketing.Client()
#     client.set_config({
#         "api_key": api_key,
#         "server": server
#     })

#     emails_openers = []
#     offset = 0
#     count = 200  # Set a higher count for each request

#     try:
#         while True:
#             response_openers = client.reports.get_campaign_open_details(
#                 campaign_id,
#                 count=count,
#                 offset=offset
#             )
            
#             # Extract email addresses from the current page of results
#             members = response_openers.get("members", [])
#             if not members:
#                 # If no members are returned, we have reached the end of the list
#                 break
            
#             for member in members:
#                 emails_openers.append(member["email_address"])
                
#             # Increment the offset for the next page
#             offset += count

#         # Create DataFrame from all collected email addresses
#         df_openers = pd.DataFrame(emails_openers, columns=["email_address"])
#         # display(df_openers)
#         return df_openers

#     except ApiClientError as error:
#         print(f"Error: {error.text}")

In [213]:
# def getClickers(api_key, server, campaign_id):
#     import mailchimp_marketing as MailchimpMarketing
#     from mailchimp_marketing.api_client import ApiClientError

#     # setup client
#     client = MailchimpMarketing.Client()
#     client.set_config({
#         "api_key": api_key,
#         "server": server
#     })

#     rows = []

#     try:
#         # 1) Get all links in the campaign
#         links_resp = client.reports.get_campaign_click_details(campaign_id)
#         links = links_resp.get("urls_clicked", [])

#         # 2) Loop through each link and get clickers
#         for link in links:
#             link_id = link["id"]
#             url = link["url"]
#             # print(f"Processing link: {url} (ID: {link_id})")

#             offset, count = 0, 1000
#             while True:
#                 resp = client.reports.get_subscribers_info(
#                     campaign_id,
#                     link_id,
#                     count=count,
#                     offset=offset
#                 )
#                 members = resp.get("members", [])

#                 for m in members:
#                     email = m.get("email_address")
#                     if email:
#                         rows.append({
#                             "email_address": email,
#                             "contact_id": m.get("contact_id"),
#                             "clicks_for_this_link": m.get("clicks"),
#                             "url": url,
#                             "link_id": link_id
#                         })

#                 if len(members) < count:
#                     break
#                 offset += count

#     except ApiClientError as error:
#         print(f"Error: {error.text}")

#     # 3) Create DataFrames
#     df_clickers = pd.DataFrame(rows)
#     # Add a check to see if the DataFrame is empty before proceeding
#     if not df_clickers.empty:
#         df_unique_emails = df_clickers[["email_address"]].drop_duplicates().sort_values("email_address").reset_index(drop=True)
        
#         # print("\n--- All Clickers (sample) ---")
#         # display(df_clickers)

#         # print("\n--- Unique Emails ---")
#         # display(df_unique_emails)

#         return df_clickers
#         # return df_unique_emails
#     else:
#         print("\nNo click data found for this campaign.")

In [214]:
# def getCampaignDetails(api_key, server, search_value):
#     import mailchimp_marketing as MailchimpMarketing
#     from mailchimp_marketing.api_client import ApiClientError
    
#     try:
#         client = MailchimpMarketing.Client()
#         client.set_config({
#             "api_key": api_key,
#             "server": server
#         })

#         response = client.searchCampaigns.search(search_value)
#         print(response) # Keep this if you want to see the raw response

#         data = response

#         # Initialize lists to collect DataFrames
#         extracted_campaign_data = []
#         all_openers_dfs = []
#         all_clickers_dfs = []
#         all_unique_clickers_dfs = []
#         all_summary_dfs = []

#         for campaign_info in data['results']:
#             campaign = campaign_info['campaign']
#             campaign_id = campaign['id']
#             campaign_title = campaign['settings']['title']

#             # 1. Collect basic campaign info
#             extracted_campaign_data.append({'id': campaign_id, 'title': campaign_title})

#             # 2. Get Openers DataFrame
#             current_openers_df = getOpeners(api_key, server, campaign_id)
#             # Add campaign_name column to the current openers DataFrame
#             current_openers_df['campaign_name'] = campaign_title
#             all_openers_dfs.append(current_openers_df) # Add to the list

#             # 3. Get Clickers DataFrame
#             current_clickers_df = getClickers(api_key, server, campaign_id)
#             # Add campaign_name column to the current clickers DataFrame
#             current_clickers_df['campaign_name'] = campaign_title
#             all_clickers_dfs.append(current_clickers_df) # Add to the list

#             # 4. Calculate counts
#             count_open = current_openers_df.shape[0] # Number of rows in openers

#             # 5. Process unique clickers
#             if not current_clickers_df.empty:
#                 # Correctly derive current_unique_clickers_df from current_clickers_df
#                 current_unique_clickers_df = current_clickers_df[["email_address"]].drop_duplicates().sort_values("email_address").reset_index(drop=True)
#                 current_unique_clickers_df['campaign_name'] = campaign_title
#                 all_unique_clickers_dfs.append(current_unique_clickers_df) # Add to the list
#                 count_click = current_clickers_df.shape[0] # Number of rows in clickers
#             else:
#                 # If clickers are empty, unique clickers is also empty
#                 all_unique_clickers_dfs.append(pd.DataFrame(columns=['email_address', 'campaign_name'])) # Append empty DF with expected columns
#                 count_click = 0

#             # 6. Create summary DataFrame for the current campaign
#             # Use 'campaign_title' as the index for clarity or add it as a column
#             # Changed index to column for easier concatenation later
#             current_summary_df = pd.DataFrame({
#                 'campaign_title': [campaign_title],
#                 'open': [count_open],
#                 'click': [count_click]
#             })
#             all_summary_dfs.append(current_summary_df) # Add to the list

#         # --- Concatenate all collected DataFrames outside the loop ---
#         df_campaign = pd.DataFrame(extracted_campaign_data)

#         # Concatenate only if lists are not empty to avoid errors on empty searches
#         df_openers = pd.concat(all_openers_dfs, ignore_index=True) if all_openers_dfs else pd.DataFrame(columns=['email_address', 'open_time', 'campaign_name'])
#         df_clickers = pd.concat(all_clickers_dfs, ignore_index=True) if all_clickers_dfs else pd.DataFrame(columns=['email_address', 'click_time', 'campaign_name'])
#         df_unique_clickers = pd.concat(all_unique_clickers_dfs, ignore_index=True) if all_unique_clickers_dfs else pd.DataFrame(columns=['email_address', 'campaign_name'])
#         df_summary = pd.concat(all_summary_dfs, ignore_index=True) if all_summary_dfs else pd.DataFrame(columns=['campaign_title', 'open', 'click'])

#         # Display results
#         display(df_campaign)

#         print("\n--- Openers ---")
#         display(df_openers)

#         print("\n--- Clickers ---")
#         display(df_clickers)

#         print("\n--- Unique Clickers ---")
#         display(df_unique_clickers)

#         print("\n--- Summary ---")
#         display(df_summary)

#         return df_campaign # Or return all DataFrames if needed
#     except ApiClientError as error:
#         print(f"Error: {error.text}")
#     except Exception as e:
#         print(f"An unexpected error occurred: {e}")